# XHS Data Cleaner (Pattern-Driven)

Noise-removal notebook for `xhs_scraped_data_clean.json`.

- Learns recurring low-value OCR lines from the input dataset
- Applies deterministic UI/comment-tail cleanup
- Preserves main post content for downstream labeling

In [ ]:
# Segment 1: Imports, file paths, and global regex patterns used by cleaner
import json
import re
import unicodedata
from collections import Counter
from pathlib import Path
from typing import Any, Dict, List, Set

# Notebook expects to run from server/scripts/xhsScrape
BASE_DIR = Path.cwd()
INPUT_PATH = BASE_DIR / "xhs_scraped_data_clean.json"
OUTPUT_PATH = BASE_DIR / "xhs_scraped_data_cleaned.json"

# Static UI/noise patterns commonly seen in OCR captures
UI_PATTERNS = [
    re.compile(r"^focus\s+on$", re.IGNORECASE),
    re.compile(r"^关注$"),
    re.compile(r"^edited\s+on\s+\d{4}-\d{2}-\d{2}$", re.IGNORECASE),
    re.compile(r"^\d+\s+comments?\s+in\s+total$", re.IGNORECASE),
    re.compile(r"^there\s+are\s+\d+\s+comments?\s+in\s+total$", re.IGNORECASE),
    re.compile(r"^show(?:\s+\d+)?\s+replies?$", re.IGNORECASE),
    re.compile(r"^reply$", re.IGNORECASE),
    re.compile(r"^author$", re.IGNORECASE),
    re.compile(r"^comment$", re.IGNORECASE),
    re.compile(r"^praise$", re.IGNORECASE),
    re.compile(r"^(?:say\s+something.*|说点什么.*|it'?s\s*a\s*wasteland.*|这是一片荒地.*)$", re.IGNORECASE),
]

DATE_LINE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:\s+[A-Za-z]+)?$", re.IGNORECASE)
SHORT_DATE_LINE = re.compile(r"^\d{2}-\d{2}(?:\s+[A-Za-z]+)?$", re.IGNORECASE)
HASHTAG = re.compile(r"#\w+")
NOISE_CODE = re.compile(r"^[A-Za-z0-9]{6,}$")
NON_CONTENT = re.compile(r"^[\d\s@QOIl|/\\._()><-]+$")
WORD_OR_HAN = re.compile(r"[A-Za-z]+|[\u4e00-\u9fff]+")
# Markers that usually indicate comment/thread tail (not main post body)
COMMENT_MARKERS = [
    re.compile(r"comments?\s+in\s+total", re.IGNORECASE),
    re.compile(r"show(?:\s+\d+)?\s+replies?", re.IGNORECASE),
    re.compile(r"\bauthor\b", re.IGNORECASE),
    re.compile(r"\breply\b", re.IGNORECASE),
]

In [ ]:
# Segment 2: Line normalization + noise classification + dynamic noise learning
def normalize_line(line: str) -> str:
    line = unicodedata.normalize("NFKC", line)
    line = line.replace("\u00a0", " ")
    line = re.sub(r"\s+", " ", line).strip()
    line = re.sub(r"\s+([,.;:!?])", r"\1", line)
    return line


def line_is_ui_noise(line: str, learned_noise: Set[str]) -> bool:
    if not line:
        return True
    lowered = line.casefold()
    if lowered in learned_noise:
        return True
    if HASHTAG.search(line):
        return True
    if NOISE_CODE.fullmatch(line) and not WORD_OR_HAN.search(line.lower()):
        return True
    if NON_CONTENT.fullmatch(line):
        return True
    if any(pat.match(line) for pat in UI_PATTERNS):
        return True
    return False


def looks_like_comment_tail_start(line: str) -> bool:
    if any(pat.search(line) for pat in COMMENT_MARKERS):
        return True
    if DATE_LINE.match(line) or SHORT_DATE_LINE.match(line):
        return True
    return False


# Learns repeated low-value lines from the current dataset itself
def learn_noise_patterns(data: Dict[str, Any]) -> Set[str]:
    counter: Counter[str] = Counter()
    post_count = 0

    for posts in data.values():
        for post in posts:
            post_count += 1
            source = str(post.get("cleaned") or post.get("raw") or "")
            seen_local = set()
            for raw_line in source.splitlines():
                line = normalize_line(raw_line)
                if not line:
                    continue
                key = line.casefold()
                if key in seen_local:
                    continue
                seen_local.add(key)
                counter[key] += 1

    learned = set()
    if post_count == 0:
        return learned

    threshold = max(2, int(post_count * 0.4))
    for line_key, freq in counter.items():
        if freq < threshold:
            continue

        tokens = WORD_OR_HAN.findall(line_key)
        if len(tokens) <= 3 and len(line_key) <= 40:
            if any(marker.search(line_key) for marker in COMMENT_MARKERS) or NON_CONTENT.fullmatch(line_key):
                learned.add(line_key)

    return learned


# Applies deterministic cleanup to one OCR text block
def clean_text_block(text: str, learned_noise: Set[str]) -> Dict[str, Any]:
    lines = [normalize_line(line) for line in text.splitlines()]
    lines = [line for line in lines if line]

    cleaned: List[str] = []
    removed: List[str] = []
    seen = set()

    comment_tail_started = False
    for line in lines:
        if line_is_ui_noise(line, learned_noise):
            removed.append(line)
            continue

        if looks_like_comment_tail_start(line):
            comment_tail_started = True

        if comment_tail_started:
            removed.append(line)
            continue

        key = line.casefold()
        if key in seen:
            removed.append(line)
            continue

        if len(WORD_OR_HAN.findall(line)) <= 1 and len(line) < 6:
            removed.append(line)
            continue

        seen.add(key)
        cleaned.append(line)

    return {
        "enhanced_cleaned": "\n".join(cleaned).strip(),
        "removed_line_count": len(removed),
        "removed_lines_sample": removed[:12],
    }

In [ ]:
# Segment 3: Dataset-level transformation and summary statistics
def transform_dataset(data: Dict[str, Any]) -> Dict[str, Any]:
    learned_noise = learn_noise_patterns(data)

    categories: Dict[str, List[Dict[str, Any]]] = {}
    post_count = 0
    removed_total = 0

    for category, posts in data.items():
        transformed_posts: List[Dict[str, Any]] = []
        for post in posts:
            source_text = str(post.get("cleaned") or post.get("raw") or "")
            result = clean_text_block(source_text, learned_noise=learned_noise)

            transformed_posts.append({
                "filename": post.get("filename"),
                "source_cleaned": source_text,
                "enhanced_cleaned": result["enhanced_cleaned"],
                "removed_line_count": result["removed_line_count"],
                "removed_lines_sample": result["removed_lines_sample"],
            })

            post_count += 1
            removed_total += result["removed_line_count"]

        categories[category] = transformed_posts

    return {
        "cleaner": "xhs_data_cleaner_v2_pattern_notebook",
        "input_path": str(INPUT_PATH),
        "output_path": str(OUTPUT_PATH),
        "stats": {
            "categories": len(categories),
            "posts": post_count,
            "removed_lines_total": removed_total,
            "learned_noise_count": len(learned_noise),
            "learned_noise_sample": sorted(list(learned_noise))[:12],
        },
        "categories": categories,
    }


# Orchestrates read -> clean -> write pipeline
def run_cleaner(input_path: Path = INPUT_PATH, output_path: Path = OUTPUT_PATH) -> Dict[str, Any]:
    with input_path.open("r", encoding="utf-8") as infile:
        data = json.load(infile)

    transformed = transform_dataset(data)

    with output_path.open("w", encoding="utf-8") as outfile:
        json.dump(transformed, outfile, ensure_ascii=False, indent=2)

    return transformed

In [ ]:
# Segment 4: Execute cleaner and print quick QA preview
result = run_cleaner()
print(f"Cleaned output saved to: {OUTPUT_PATH}")
print(result["stats"])

for category, posts in result["categories"].items():
    safe_category = category.encode('ascii', 'ignore').decode('ascii') or '<non-ascii-category>'
    print(f"\n[{safe_category}]")
    for post in posts[:2]:
        snippet = post['enhanced_cleaned'][:300].encode('ascii', 'ignore').decode('ascii')
        print(f"- {post['filename']}\n{snippet}\n")